# Masking invariant sites from multi-sequence alignment 
### This script generates a BED file for use with `augur mask`
### Requires VCF from aligned fasta.  

Use [snp-sites](https://github.com/sanger-pathogens/snp-sites) to generate VCF from alignment file: 
    
    `snp-sites -v -o snps.vcf aligned.fasta`


In [1]:
import pandas as pd 
import numpy as np
import re 

from pysam import VariantFile
from random import sample

vcf_path = "../monkeypox-build/results/hmpxv1/snps.vcf"
bed_file_path = "../monkeypox-build/config/mask.bed"
bed_file_out = "../monkeypox-build/config/to_mask.bed"
stats_out = "../out/masking/masked_300_HQ.txt"

In [2]:
# read in VCF
vcf = VariantFile(vcf_path)

# get alignment length
alignment_length = 0 
for rec in vcf.header.records:
    match = (re.compile("\d{5}\d")).search(str(rec))
    if match:
        alignment_length = int(match.group())

# get variant sites, store in snps[] 
snps = [] 
for snp in vcf.fetch():
    snps.append(snp.pos)

#get invariant sites, store in invar[] 
invar = []
for i in range(1, (alignment_length+1)):
    if i not in snps:
        invar.append(i)
    
#get fraction of variants vs invariants 
var_frac = round((len(snps)/alignment_length),3)
mono_frac = 1 - var_frac


## sample a random 90% of invar[] to remove
prune = int(0.9*alignment_length) 
remove = sample(invar,prune)


### Append invariants to BED file
Note: BED text format uses 0-based positions + half-open notation range 
      The VCF position POS is 1-based 

In [3]:
# import existing mask.bed file
bed = pd.read_csv(bed_file_path, delimiter='\t')
bed_out = pd.read_csv(bed_file_path, delimiter='\t')

# append sites-to-mask
bed_out = bed_out.reindex(list(range(0, (len(bed)+prune)))).reset_index(drop=True) 
bed_out.loc[:,'Chrom'] = 'chr'

chrom_start = [x - 1 for x in remove]
bed_out.loc[len(bed):len(bed)+prune, 'ChromStart'] = chrom_start
 
bed_out.loc[len(bed):len(bed)+prune, 'ChromEnd'] = remove

# add description 
bed_out.loc[len(bed):len(bed)+prune, 'Comment'] = 'invariant site'

# float to int
bed_out['ChromStart'] = bed_out['ChromStart'].astype(int)
bed_out['ChromEnd'] = bed_out['ChromEnd'].astype(int)

In [4]:
bed_out.to_csv(bed_file_out, sep ='\t', index=False)

In [5]:
# report basic stats

print("before pruning:")
print("alignment length:", alignment_length)
print("variable sites:", len(snps), '(',var_frac,'%)')
print("invariant sites:", len(invar), '(', mono_frac,'%)')
print("\n")
print("after pruning:")
print("alignment length:", (len(invar) - prune) + len(snps), '(removed', prune, 'sites)' )  
print("variable sites:", len(snps),
      '(',round(  (len(snps) / ((len(invar) - prune)+len(snps))) ,3),'%)')
print("invariant sites:", len(invar) - prune, 
      '(', round( (len(invar)-prune) / ((len(invar) - prune)+len(snps)),3  )  ,'%)')

before pruning:
alignment length: 197209
variable sites: 1527 ( 0.008 %)
invariant sites: 195682 ( 0.992 %)


after pruning:
alignment length: 19721 (removed 177488 sites)
variable sites: 1527 ( 0.077 %)
invariant sites: 18194 ( 0.923 %)


In [28]:
# write stats to txt file  

l1 = "\n # before pruning:"
l2 = "\n # alignment length:" + str(alignment_length) 
l3 = "\n # variable sites:" + str(len(snps)) + '(' + str(var_frac)+ '%)'
l4 = "\n # invariant sites:" + str(len(invar)) + '(' + str(mono_frac) + '%)'
l5 = "\n # after pruning:"
l6 = "\n # alignment length:" + str((len(invar) - prune) + len(snps)) + '(removed' + str( prune) + 'sites)' 
l7 = "\n # variable sites:" +  str(len(snps)) +'(' + str(round(  (len(snps) / ((len(invar) - prune)+len(snps))) ,3)) + '%)'
l8 = "\n # invariant sites:" + str(len(invar) - prune) + '(' + str(round( (len(invar)-prune) / ((len(invar) - prune)+len(snps)),3  ))  + '%)'


with open(stats_out, 'a') as file:
    file.writelines([l1, l2, l3, l4, l5, l6, l7, l8])